In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 18.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
import joblib
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300


# LOAD AND PREPARE DATA

df = pd.read_csv('/content/drive/MyDrive/final_dataset.csv')
print(f"Loaded {len(df)} samples")

df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['season'] = df['date'].dt.quarter
df['day_of_year'] = df['date'].dt.dayofyear
df['year'] = df['date'].dt.year
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['is_winter'] = df['month'].isin([12, 1, 2]).astype(int)
df['is_spring'] = df['month'].isin([3, 4, 5]).astype(int)
df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
df['is_autumn'] = df['month'].isin([9, 10, 11]).astype(int)

if 'VV_backscatter' in df.columns and 'VH_backscatter' in df.columns:
    epsilon = 1e-10
    df['VV_VH_ratio'] = df['VV_backscatter'] / (df['VH_backscatter'] + epsilon)
    df['cross_pol_ratio'] = df['VH_backscatter'] / (df['VV_backscatter'] + epsilon)
    df['roughness_proxy'] = np.abs(df['VV_backscatter'] - df['VH_backscatter'])
    df['backscatter_diff'] = df['VV_backscatter'] - df['VH_backscatter']
    df['backscatter_sum'] = df['VV_backscatter'] + df['VH_backscatter']
    df['backscatter_mean'] = (df['VV_backscatter'] + df['VH_backscatter']) / 2
    df['surface_roughness_index'] = np.abs(df['VV_backscatter'] - df['VH_backscatter']) / (np.abs(df['VV_backscatter']) + epsilon)
    df['roughness_variance'] = (df['VV_backscatter'] - df['VH_backscatter']) ** 2
    df['normalized_roughness'] = df['backscatter_diff'] / (df['backscatter_sum'] + epsilon)
    df['flow_velocity_proxy'] = df['roughness_proxy'] * df['VV_VH_ratio']
    df['turbulence_indicator'] = df['roughness_variance'] * df['surface_roughness_index']
    vv_min, vv_max = df['VV_backscatter'].min(), df['VV_backscatter'].max()
    vh_min, vh_max = df['VH_backscatter'].min(), df['VH_backscatter'].max()
    df['VV_norm'] = (df['VV_backscatter'] - vv_min) / (vv_max - vv_min)
    df['VH_norm'] = (df['VH_backscatter'] - vh_min) / (vh_max - vh_min)
if 'sst' in df.columns:
    df['water_temp'] = df['sst']
    df['temp_cold'] = (df['water_temp'] < df['water_temp'].quantile(0.25)).astype(int)
    df['temp_warm'] = (df['water_temp'] > df['water_temp'].quantile(0.75)).astype(int)
    df['temp_moderate'] = ((df['water_temp'] >= df['water_temp'].quantile(0.25)) &
                           (df['water_temp'] <= df['water_temp'].quantile(0.75))).astype(int)
    df['water_temp_squared'] = df['water_temp'] ** 2
    df['water_temp_log'] = np.log1p(df['water_temp'] - df['water_temp'].min() + 1)
    df['water_temp_abs'] = np.abs(df['water_temp'])
    df['temp_range'] = df['water_temp'] - df['water_temp'].mean()

if 'chlorophyll_a' in df.columns:
    df['log_chlorophyll'] = np.log1p(df['chlorophyll_a'])
    df['chlorophyll_squared'] = df['chlorophyll_a'] ** 2
    df['chlorophyll_cubed'] = df['chlorophyll_a'] ** 3
    df['chl_low'] = (df['chlorophyll_a'] < df['chlorophyll_a'].quantile(0.33)).astype(int)
    df['chl_medium'] = ((df['chlorophyll_a'] >= df['chlorophyll_a'].quantile(0.33)) &
                        (df['chlorophyll_a'] < df['chlorophyll_a'].quantile(0.67))).astype(int)
    df['chl_high'] = (df['chlorophyll_a'] >= df['chlorophyll_a'].quantile(0.67)).astype(int)
    df['productivity_index'] = df['log_chlorophyll'] * df['water_temp'] if 'water_temp' in df.columns else df['log_chlorophyll']

if 'VV_backscatter' in df.columns and 'water_temp' in df.columns:
    df['flow_temp_interaction'] = df['roughness_proxy'] * df['water_temp']
    df['VV_temp_interaction'] = df['VV_backscatter'] * df['water_temp']
    df['VH_temp_interaction'] = df['VH_backscatter'] * df['water_temp']
    df['roughness_temp_interaction'] = df['surface_roughness_index'] * df['water_temp']

if 'VH_backscatter' in df.columns and 'chlorophyll_a' in df.columns:
    df['VH_chl_interaction'] = df['VH_backscatter'] * df['chlorophyll_a']
    df['roughness_chl_interaction'] = df['roughness_proxy'] * df['log_chlorophyll']

if 'water_temp' in df.columns and 'chlorophyll_a' in df.columns:
    df['temp_chl_interaction'] = df['water_temp'] * df['chlorophyll_a']
    df['temp_chl_ratio'] = df['water_temp'] / (df['chlorophyll_a'] + epsilon)


if 'roughness_proxy' in df.columns and 'VV_VH_ratio' in df.columns:
    df['river_energy_index'] = df['roughness_proxy'] * df['VV_VH_ratio']
    df['sediment_transport_proxy'] = df['roughness_variance'] * df['VV_VH_ratio']
    df['mixing_indicator'] = df['turbulence_indicator'] / (df['backscatter_mean'] + epsilon)


# FEATURE SELECTION FOR RIVERS

temporal_features = ['month_cos', 'season', 'is_winter', 'is_spring', 'is_summer', 'is_autumn']
sar_features = ['VV_VH_ratio', 'roughness_proxy', 'backscatter_diff', 'roughness_variance', 'flow_velocity_proxy', 'turbulence_indicator', 'VV_norm', 'VH_norm']
water_quality_features = ['water_temp', 'temp_cold', 'water_temp_squared', 'chl_low', 'chl_medium', 'chl_high', 'productivity_index']
interaction_features = ['flow_temp_interaction', 'VV_temp_interaction', 'roughness_temp_interaction', 'VH_chl_interaction','temp_chl_interaction', 'temp_chl_ratio']
river_dynamics_features = ['river_energy_index', 'sediment_transport_proxy', 'mixing_indicator']
all_river_features = (temporal_features + sar_features + water_quality_features +
                      interaction_features + river_dynamics_features)

# Filter for available features
available_features = [f for f in all_river_features if f in df.columns]
feature_coverage = {f: df[f].notna().sum() / len(df) * 100 for f in available_features}
good_features = [f for f, cov in feature_coverage.items() if cov >= 50]
sorted_features = sorted(feature_coverage.items(), key=lambda x: x[1], reverse=True)
for feat, cov in sorted_features[:15]:
    if feat in good_features:
        print(f"  {feat:35s}: {cov:5.1f}%")

# PREPARE DATASETS

df_valid = df[df['mp_measurement'].notna()].copy()
X = df_valid[good_features].copy()
y = df_valid['mp_measurement'].copy()

print(f"Feature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")
print(f"MP concentration range: {y.min():.3f} to {y.max():.3f}")

# Train-Test Split (BEFORE any preprocessing to prevent leakage)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)
print(f"\nTraining samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Fit preprocessing ONLY on training data
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=good_features,
    index=X_train.index
)
X_test_imputed = pd.DataFrame(
    imputer.transform(X_test),
    columns=good_features,
    index=X_test.index
)

X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)


# PART 2: OPTUNA HYPERPARAMETER TUNING

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.01),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'random_state': 42,
        'n_jobs': -1
    }

    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('model', RandomForestRegressor(**params))
    ])

    cv_scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=5, scoring='r2', n_jobs=-1
    )

    return cv_scores.mean()

# Create and run Optuna study
study = optuna.create_study(
    direction='maximize',
    study_name='river_mp_rf_optimization'
)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print(f"Best R² Score: {study.best_value:.4f}")
print(f"\nBest Hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key:25s}: {value}")

# PART 3: TRAIN FINAL MODEL

rf_model = RandomForestRegressor(**study.best_params)
rf_model.fit(X_train_imputed, y_train)

# Predictions
y_pred_train = rf_model.predict(X_train_imputed)
y_pred_test = rf_model.predict(X_test_imputed)

# Metrics
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\nModel Performance:")
print(f"  R² (train):  {r2_train:.4f}")
print(f"  R² (test):   {r2_test:.4f}")
print(f"  MAE:         {mae:.4f}")
print(f"  RMSE:        {rmse:.4f}")

# Cross-validation
print("\n=== Cross-Validation Results ===")
final_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('model', RandomForestRegressor(**study.best_params))
])

cv_scores = cross_val_score(
    final_pipeline, X_train, y_train,
    cv=5, scoring='r2', n_jobs=-1
)
print(f"CV R² Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Feature Importance Analysis
importance_df = pd.DataFrame({
    'feature': good_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== Most Important Features ===")
for idx, row in importance_df.head(15).iterrows():
    print(f"  {row['feature']:35s}: {row['importance']:.4f}")

# PART 4: VISUALIZATIONS

# Feature Importance
plt.figure(figsize=(10, 8))
top_features = importance_df.head(20)
plt.barh(range(len(top_features)), top_features['importance'],
         color='steelblue', alpha=0.8, edgecolor='black', linewidth=0.5)
plt.yticks(range(len(top_features)), top_features['feature'], fontsize=10)
plt.gca().invert_yaxis()
plt.xlabel('Importance Score', fontweight='bold', fontsize=12)
plt.title('River MP Model - [eature Importances',
          fontweight='bold', fontsize=14)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('river_feature_importance.png', dpi=300, bbox_inches='tight')
print("Saved: river_feature_importance.png")
plt.close()

# Cross-Validation Scores
plt.figure(figsize=(8, 6))
plt.bar(range(len(cv_scores)), cv_scores, alpha=0.8, color='teal',
        edgecolor='black', linewidth=1)
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--',
            linewidth=2, label=f'Mean: {cv_scores.mean():.3f}')
plt.xlabel('Fold', fontweight='bold', fontsize=12)
plt.ylabel('R² Score', fontweight='bold', fontsize=12)
plt.title('River MP Model - 5-Fold Cross-Validation Results',
          fontweight='bold', fontsize=14)
plt.xticks(range(len(cv_scores)), [f'Fold {i+1}' for i in range(len(cv_scores))])
plt.grid(True, alpha=0.3, axis='y')
plt.legend(fontsize=11)
plt.tight_layout()
plt.savefig('river_cv_scores.png', dpi=300, bbox_inches='tight')
print("Saved: river_cv_scores.png")
plt.close()

# PART 5: SAVE MODEL AND OUTPUTS

# Save model
joblib.dump(rf_model, 'river_mp_model.pkl')
joblib.dump(imputer, 'river_imputer.pkl')
joblib.dump(scaler, 'river_scaler.pkl')

# Save feature importance
importance_df.to_csv('river_feature_importance.csv', index=False)
print("Saved: river_feature_importance.csv")

# Save model summary
summary = f"""
RIVER MICROPLASTICS PREDICTION MODEL - SUMMARY
===============================================

MODEL PERFORMANCE:
  - Training R²:        {r2_train:.4f}
  - Test R²:            {r2_test:.4f}
  - CV R² (5-fold):     {cv_scores.mean():.4f} ± {cv_scores.std()*2:.4f}
  - MAE:                {mae:.4f}
  - RMSE:               {rmse:.4f}

DATASET:
  - Total samples:      {len(df_valid)}
  - Training samples:   {len(X_train)}
  - Test samples:       {len(X_test)}
  - Number of features: {len(good_features)}

TOP 10 FEATURES:
"""
for idx, row in importance_df.head(10).iterrows():
    summary += f"  {idx+1:2d}. {row['feature']:30s} - {row['importance']:.4f}\n"

for key, value in study.best_params.items():
    summary += f"  {key:25s}: {value}\n"

print(f"\nModel trained with {len(good_features)} river-specific features")
print(f"Test R²: {r2_test:.4f} | MAE: {mae:.4f} | RMSE: {rmse:.4f}")
print("="*70)

Loaded 341 samples
  month_cos                          : 100.0%
  season                             : 100.0%
  is_winter                          : 100.0%
  is_spring                          : 100.0%
  is_summer                          : 100.0%
  is_autumn                          : 100.0%
  VV_VH_ratio                        : 100.0%
  roughness_proxy                    : 100.0%
  backscatter_diff                   : 100.0%
  roughness_variance                 : 100.0%
  flow_velocity_proxy                : 100.0%
  turbulence_indicator               : 100.0%
  VV_norm                            : 100.0%
  VH_norm                            : 100.0%
  water_temp                         : 100.0%
Feature matrix shape: (341, 30)
Target variable shape: (341,)
MP concentration range: 0.000 to 2.565

Training samples: 272
Test samples: 69


  0%|          | 0/50 [00:00<?, ?it/s]

Best R² Score: 0.7694

Best Hyperparameters:
  n_estimators             : 400
  max_depth                : 30
  min_samples_split        : 17
  min_samples_leaf         : 2
  max_features             : sqrt
  min_impurity_decrease    : 0.0009120220507649182
  bootstrap                : False

Model Performance:
  R² (train):  0.8483
  R² (test):   0.8010
  MAE:         0.0528
  RMSE:        0.1696

=== Cross-Validation Results ===
CV R² Score: 0.7675 (+/- 0.2444)

=== Most Important Features ===
  flow_temp_interaction              : 0.1491
  roughness_temp_interaction         : 0.1121
  water_temp                         : 0.1008
  water_temp_squared                 : 0.0866
  productivity_index                 : 0.0814
  turbulence_indicator               : 0.0738
  temp_chl_interaction               : 0.0505
  mixing_indicator                   : 0.0438
  VH_chl_interaction                 : 0.0403
  roughness_variance                 : 0.0337
  backscatter_diff                   : 